### Imports 

In [1]:
# ML Imports 
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import json
import os

# AWS Imports 
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer
from sagemaker import get_execution_role
import sagemaker
import boto3

### 1. Load & Prep Data

The <a href='https://homepages.inf.ed.ac.uk/imurray2/teaching/oranges_and_lemons/'>Fruits Dataset</a> was originally created by Dr. Iain Murray from University of Edinburgh and extended more recently by the University of Michigan. It is a simple multi-class dataset with 4 columns (features) and 4 classes (fruits). The 4 classes are apple, orange, mandarin and lemon. The four features are mass, width, height and color score of the fruit.

The color score feature maps to a color and its intensity in the color spectrum (0 - 1) scale. <br><br>
<table align="left" style="width:50%">
    <tr>
        <th>Color</th>
        <th>Range</th>
    </tr>
    <tr>
        <td>Red</td>
        <td>0.85 - 1.00</td>
    </tr>
    <tr>
        <td>Orange</td>
        <td>0.75 - 0.85</td>
    </tr>
    <tr>
        <td>Yellow</td>
        <td>0.65 - 0.75</td>
    </tr>
    <tr>
        <td>Green</td>
        <td>0.45 - 0.65</td>
    </tr>
</table>

In [2]:
df = pd.read_csv('./DATA/fruits.csv', names=['class', 'mass', 'width', 'height', 'color_score'])

In [3]:
df.sample(10)

,class,mass,width,height,color_score
15,apple,156,7.7,7.1,0.69
9,apple,172,7.4,7.0,0.89
3,mandarin,86,6.2,4.7,0.80
39,orange,144,6.8,7.4,0.75
27,orange,204,7.5,9.2,0.77
24,orange,342,9.0,9.4,0.75
5,mandarin,80,5.8,4.3,0.77
13,apple,164,7.3,7.7,0.70
35,orange,150,7.1,7.9,0.75
7,mandarin,76,5.8,4.0,0.81


In [4]:
df.shape

(59, 5)

In [5]:
df['class'].unique().tolist()

['apple', 'mandarin', 'orange', 'lemon']

In [6]:
X = df[['mass', 'width', 'height', 'color_score']]
y = df['class']

##### Encode the classes into numerical values using Sklearn's LabelEncoder

In [7]:
label_encoder = LabelEncoder()
label_encoder.fit(['apple', 'orange', 'mandarin', 'lemon'])
y = label_encoder.transform(y)

In [8]:
y

array([0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

##### Split X, y into train and test sets

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [10]:
X_train.shape

(44, 4)

In [11]:
X_test.shape

(15, 4)

##### Scale feature columns using Sklearn's MinMaxScaler

In [12]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
X_train[0]

array([0.32142857, 0.32352941, 0.50769231, 1.        ])

In [14]:
y_train[0]

0

In [15]:
print(type(X_train), type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


##### Combine Scaled X & y into Train and Test DataFrames 

In [16]:
X_train = pd.DataFrame(X_train, columns=['mass', 'width', 'height', 'color_score'])
y_train = pd.DataFrame(y_train, columns=['class'])
train_df = pd.concat([y_train, X_train], axis=1)
train_df.head()

,class,mass,width,height,color_score
0,0,0.321429,0.323529,0.507692,1.000000
1,1,0.421429,0.411765,0.969231,0.323529
2,0,0.364286,0.382353,0.584615,0.970588
3,0,0.278571,0.352941,0.476923,0.852941
4,1,0.192857,0.058824,0.646154,0.352941


In [17]:
X_test = pd.DataFrame(X_test, columns=['mass', 'width', 'height', 'color_score'])
y_test = pd.DataFrame(y_test, columns=['class'])
test_df = pd.concat([y_test, X_test], axis=1)
test_df.head()

,class,mass,width,height,color_score
0,1,0.142857,0.058824,0.538462,0.382353
1,3,0.371429,0.529412,0.646154,0.588235
2,0,0.314286,0.441176,0.569231,0.323529
3,1,0.157143,0.058824,0.676923,0.441176
4,3,0.457143,0.500000,0.800000,0.529412


##### Create a DataFrame for Batch Inference without the Class column

In [42]:
batch_test_df = test_df.drop(['class'], axis=1)
batch_test_df.head()

,mass,width,height,color_score
0,0.142857,0.058824,0.538462,0.382353
1,0.371429,0.529412,0.646154,0.588235
2,0.314286,0.441176,0.569231,0.323529
3,0.157143,0.058824,0.676923,0.441176
4,0.457143,0.500000,0.800000,0.529412


##### Write Train & Test Sets to Local Directory

In [43]:
train_df.to_csv('./DATA/train.csv', header=False, index=False)
test_df.to_csv('./DATA/test.csv', header=False, index=False)
batch_test_df.to_csv('./DATA/batch_test.csv', header=False, index=False)

### 2. Essentials 

In [44]:
# Get the Execution Role for this Notebook and AWS Session to create Clients
sagemaker_execution_role = get_execution_role()
print('Role = {}'.format(sagemaker_execution_role))
session = boto3.Session()

# Clients and Resources
s3 = session.resource('s3')
sagemaker_session = sagemaker.Session()

BUCKET = 'sagemaker-demo-892313895307' # USE YOUR ACCOUNT ID OR INITIALS AS SUFFIX
PREFIX = 'fruits-clf'

Role = arn:aws:iam::892313895307:role/SageMakerPersonalizeForecastRole


##### Upload Train & Test Sets to S3 and Create Pointers to Data

In [45]:
s3.create_bucket(Bucket=BUCKET)

s3.Bucket(name='sagemaker-demo-892313895307')

In [46]:
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'train/train.csv')).upload_file('./DATA/train.csv')
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'test/test.csv')).upload_file('./DATA/test.csv')
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'batch_test/batch_test.csv')).upload_file('./DATA/batch_test.csv')

##### Create Pointers to the uploaded files 

In [22]:
train_set_location = 's3://{}/{}/train/'.format(BUCKET, PREFIX)
test_set_location = 's3://{}/{}/test/'.format(BUCKET, PREFIX)

In [23]:
print(train_set_location)
print(test_set_location)

s3://sagemaker-demo-892313895307/fruits-clf/train/
s3://sagemaker-demo-892313895307/fruits-clf/test/


In [24]:
train_set_pointer = sagemaker.s3_input(s3_data=train_set_location, content_type='csv')
test_set_pointer = sagemaker.s3_input(s3_data=test_set_location, content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [25]:
print(json.dumps(train_set_pointer.__dict__, indent=2))

{
  "config": {
    "DataSource": {
      "S3DataSource": {
        "S3DataType": "S3Prefix",
        "S3Uri": "s3://sagemaker-demo-892313895307/fruits-clf/train/",
        "S3DataDistributionType": "FullyReplicated"
      }
    },
    "ContentType": "csv"
  }
}


### 3. Train a Model using SageMaker + Builtin XgBoost Algorithm

In [26]:
container_uri = get_image_uri(session.region_name, 'xgboost', '1.0-1')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [27]:
xgb = sagemaker.estimator.Estimator(image_name=container_uri,
                                    role=sagemaker_execution_role, 
                                    train_instance_count=2, 
                                    train_instance_type='ml.m5.large',
                                    output_path='s3://{}/{}/model-artifacts'.format(BUCKET, PREFIX),
                                    sagemaker_session=sagemaker_session,
                                    base_job_name='fruits-clf-xgboost')

xgb.set_hyperparameters(objective='multi:softmax',
                        num_class=4,
                        num_round=100)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [28]:
xgb.fit({'train': train_set_pointer, 'validation': test_set_pointer})

2020-10-21 19:34:32 Starting - Starting the training job...
2020-10-21 19:34:35 Starting - Launching requested ML instances......
2020-10-21 19:35:49 Starting - Preparing the instances for training......
2020-10-21 19:36:47 Downloading - Downloading input data...
2020-10-21 19:37:20 Training - Downloading the training image.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[19:37:39] 44x4 matrix with 176 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of CSV inpu

INFO:RabitTracker:[42]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[43]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[44]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[45]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[46]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[47]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[48]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[49]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[50]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[51]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[52]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[53]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTracker:[54]#011train-merror:0.00000#011validation-merror:0.13333
INFO:RabitTr

### 4. Host the Trained Model as a SageMaker Endpoint (using Estimator object)

In [30]:
xgb_predictor = xgb.deploy(initial_instance_count=2,
                           instance_type='ml.m5.large')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Using already existing model: fruits-clf-xgboost-2020-10-21-19-34-32-454


---------------!

### 5. Real Time Inference from the Endpoint

In [31]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

In [32]:
test_df = pd.read_csv('./DATA/test.csv', names=['class', 'mass', 'width', 'height', 'color_score'])

In [33]:
test_df.head()

,class,mass,width,height,color_score
0,1,0.142857,0.058824,0.538462,0.382353
1,3,0.371429,0.529412,0.646154,0.588235
2,0,0.314286,0.441176,0.569231,0.323529
3,1,0.157143,0.058824,0.676923,0.441176
4,3,0.457143,0.500000,0.800000,0.529412


In [34]:
X = test_df.sample(1)
X

,class,mass,width,height,color_score
12,0,0.342857,0.382353,0.553846,0.970588


In [35]:
X = X.values[0]
X[1:]

array([0.34285714, 0.38235294, 0.55384615, 0.97058824])

In [36]:
payload = X[1:]

In [37]:
%%time

predicted_class = xgb_predictor.predict(payload).decode('utf-8')

CPU times: user 18.9 ms, sys: 604 µs, total: 19.5 ms
Wall time: 1.16 s


In [38]:
predicted_class_label = label_encoder.inverse_transform([int(float(predicted_class))])
print('Predicted Class = {} ==> {}'.format(predicted_class, predicted_class_label[0]))

Predicted Class = 0.0 ==> apple


### 5. Evaluate (Batch Inference using Endpoint) 

In [39]:
predictions = []
expected = []
correct = 0
for row in test_df.values:
    expected_class = row[0]
    payload = row[1:]
    predicted_class = xgb_predictor.predict(payload).decode('utf-8')
    if predicted_class == str(expected_class):
        correct += 1
    predictions.append(predicted_class)
    expected.append(expected_class)

In [40]:
print('Accuracy = {:.2f}%'.format(correct/len(predictions) * 100))

Accuracy = 86.67%


##### Confusion Matrix

In [41]:
expected = pd.Series(np.array(expected))
predictions = pd.Series(np.array(predictions))
pd.crosstab(expected, predictions, rownames=['Actual'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,2.0,3.0,All
Actual,,,,,
0.0,4,1,0,0,5
1.0,0,2,0,1,3
2.0,0,0,1,0,1
3.0,0,0,0,6,6
All,4,3,1,7,15
